# Workflow

OR Write a single script that starts with call to FilteredMerfishDataset, extract animals with that behavior and sex, and then run a CV.

In [5]:
import pandas as pd
import json
import time

import pytorch_lightning as pl
from omegaconf import DictConfig, OmegaConf
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from torch.utils.data import random_split
from torch_geometric.data import DataLoader

from spatial.merfish_dataset import FilteredMerfishDataset, MerfishDataset
from spatial.models.monet_ae import MonetAutoencoder2D, TrivialAutoencoder
from spatial.train import train
from spatial.predict import test

import torch

import hydra
from hydra.experimental import compose, initialize

In [6]:
behaviors = ["Parenting", "Virgin Parenting", "Naive"]
sexes = ["Female"]

In [7]:
with open('animal_id.json') as json_file:
    animals = json.load(json_file)

In [ ]:
loss_dict = {}
time_dict = {}
loss_excitatory_dict = {}
loss_inhibitory_dict = {}

for behavior in behaviors:
    for sex in sexes:
        try:
            animal_list = animals[behavior][sex]
        except KeyError:
            continue
        behavior = [behavior]
        sex = [sex]
        # print(behavior, sex, animal_list)
        for animal in animal_list:
            start = time.time()
            with initialize(config_path="../config"):
                cfg_from_terminal = compose(config_name="config")
                # update the behavior to get the model of interest
                OmegaConf.update(cfg_from_terminal, "datasets.dataset.behaviors", behavior)
                OmegaConf.update(cfg_from_terminal, "datasets.dataset.sexes", sex)
                OmegaConf.update(cfg_from_terminal, "datasets.dataset.test_animal", animal)
                model = train(cfg_from_terminal)
                output = test(cfg_from_terminal)
                trainer, l1_losses, inputs, gene_expressions, celltypes, test_results = output
                MAE = test_results[0]['test_loss: mae_response']
                excitatory_cells = (celltypes == 6).nonzero(as_tuple=True)[0]
                MAE_excitatory = torch.abs(torch.index_select((gene_expressions-inputs)[excitatory_cells], 1, torch.tensor(model.responses))).mean().item()
                inhibitory_cells = (celltypes == 7).nonzero(as_tuple=True)[0]
                MAE_inhibitory = torch.abs(torch.index_select((gene_expressions-inputs)[inhibitory_cells], 1, torch.tensor(model.responses))).mean().item()
            end = time.time()
            time_dict[f"{sex}_{behavior}_{animal}"] = end-start
            loss_dict[f"{sex}_{behavior}_{animal}"] = MAE
            loss_excitatory_dict[f"{sex}_{behavior}_{animal}"] = MAE_excitatory
            loss_inhibitory_dict[f"{sex}_{behavior}_{animal}"] = MAE_inhibitory
            
            with open("deepST_MAE.json", "w") as outfile:
                json.dump(loss_dict, outfile, indent=4)

            with open("deepST_time.json", "w") as outfile:
                json.dump(time_dict, outfile, indent=4)
                
            with open("deepST_MAE_excitatory.json", "w") as outfile:
                json.dump(loss_excitatory_dict, outfile, indent=4)
                
            with open("deepST_MAE_inhibitory.json", "w") as outfile:
                json.dump(loss_inhibitory_dict, outfile, indent=4)

/home/roko/.cache/pypoetry/virtualenvs/spatial-G_n0JvVf-py3.8/lib/python3.8/site-packages/hydra/experimental/initialize.py:35: UserWarning: hydra.experimental.initialize() is no longer experimental. Use hydra.initialize()
  deprecation_warning(
/home/roko/.cache/pypoetry/virtualenvs/spatial-G_n0JvVf-py3.8/lib/python3.8/site-packages/hydra/experimental/compose.py:18: UserWarning: hydra.experimental.compose() is no longer experimental. Use hydra.compose()
  deprecation_warning(
/home/roko/.cache/pypoetry/virtualenvs/spatial-G_n0JvVf-py3.8/lib/python3.8/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'config': Defaults list is missing `_self_`. See https://hydra.cc/docs/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)
/home/roko/.cache/pypoetry/virtualenvs/spatial-G_n0JvVf-py3.8/lib/python3.8/site-packages/hydra/core/default_element.py:122: UserWarning: In 'predict/default': Usage of deprecated keyword in package head

Original Data (1027848, 170)
Filtered Data (86902, 170)
/home/roko/spatial/data/raw/merfish_messi.hdf5


/home/roko/.cache/pypoetry/virtualenvs/spatial-G_n0JvVf-py3.8/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:286: LightningDeprecationWarning: Passing `Trainer(accelerator='dp')` has been deprecated in v1.5 and will be removed in v1.7. Use `Trainer(strategy='dp')` instead.
  rank_zero_deprecation(
/home/roko/.cache/pypoetry/virtualenvs/spatial-G_n0JvVf-py3.8/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:147: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=True)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=True)`.
  rank_zero_deprecation(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]

  | Name            | Type                    | Params
------------------------------------------------------------
0 | encoder_n

Validation sanity check: 0it [00:00, ?it/s]

/home/roko/.cache/pypoetry/virtualenvs/spatial-G_n0JvVf-py3.8/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/roko/.cache/pypoetry/virtualenvs/spatial-G_n0JvVf-py3.8/lib/python3.8/site-packages/pytorch_lightning/overrides/data_parallel.py:97: UserWarning: Could not determine on which device the inputs are. When using DataParallel (strategy='dp'), be aware that in case you are using self.device in your code, it will reference only the root device.
  rank_zero_warn(
/home/roko/.cache/pypoetry/virtualenvs/spatial-G_n0JvVf-py3.8/lib/python3.8/site-packages/pytorch_lightning/utilities/data.py:59: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch 

...

Epoch 599, global step 16799: val_loss reached 0.19934 (best 0.19934), saving model to "/home/roko/spatial/output/lightning_logs/checkpoints/MonetAutoencoder2D/MonetAutoencoder2D__155__[200, 200]__155__3__['Female']__['Naive']__0.001__deepST_CV_4.ckpt" as top True
FIT Profiler Report

Action                             	|  Mean duration (s)	|Num calls      	|  Total time (s) 	|  Percentage %   	|
--------------------------------------------------------------------------------------------------------------------------------------
Total                              	|  -              	|_              	|  1839.1         	|  100 %          	|
--------------------------------------------------------------------------------------------------------------------------------------
run_training_epoch                 	|  3.0639         	|600            	|  1838.3         	|  99.961         	|
run_training_batch                 	|  0.085331       	|16800          	|  1433.6         	|  77.951      

Original Data (1027848, 170)
Filtered Data (205348, 170)
/home/roko/spatial/data/raw/merfish_messi.hdf5


/home/roko/.cache/pypoetry/virtualenvs/spatial-G_n0JvVf-py3.8/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:286: LightningDeprecationWarning: Passing `Trainer(accelerator='dp')` has been deprecated in v1.5 and will be removed in v1.7. Use `Trainer(strategy='dp')` instead.
  rank_zero_deprecation(
/home/roko/.cache/pypoetry/virtualenvs/spatial-G_n0JvVf-py3.8/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:147: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=True)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=True)`.
  rank_zero_deprecation(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]
/home/roko/.cache/pypoetry/virtualenvs/spatial-G_n0JvVf-py3.8/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.p

Testing: 0it [00:00, ?it/s]

/home/roko/.cache/pypoetry/virtualenvs/spatial-G_n0JvVf-py3.8/lib/python3.8/site-packages/pytorch_lightning/overrides/data_parallel.py:97: UserWarning: Could not determine on which device the inputs are. When using DataParallel (strategy='dp'), be aware that in case you are using self.device in your code, it will reference only the root device.
  rank_zero_warn(


## Graph Organized by Animal ID Verification

In order for a specific animal to be held out for testing, we need to understand how exactly the slices get stored before graph construction. We can accomplish this by putting a break point in unique_slices, and observing what happens and assuring it's not random. We have tried fixing unique slices, BUT CONFIRMATION IS NEEDED.